In [1]:
!apt-get install python3-pypdf2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  python3-pypdf2
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 49.2 kB of archives.
After this operation, 263 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-pypdf2 all 1.26.0-2 [49.2 kB]
Fetched 49.2 kB in 0s (102 kB/s)
Selecting previously unselected package python3-pypdf2.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../python3-pypdf2_1.26.0-2_all.deb ...
Unpacking python3-pypdf2 (1.26.0-2) ...
Setting up python3-pypdf2 (1.26.0-2) ...


In [2]:
!pip install -U -q PyDrive

In [3]:
!pip install sentence-transformers

     |████████████████████████████████| 92kB 4.5MB/s 
     |████████████████████████████████| 2.5MB 18.8MB/s 
     |████████████████████████████████| 1.2MB 32.8MB/s 
     |████████████████████████████████| 901kB 39.1MB/s 
     |████████████████████████████████| 3.3MB 34.0MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=2ef7accb137cfd6814ba5746cfdbb007c012de9033629bf351b441cd87386bf9
  Stored in directory: /root/.cache/pip/wheels/38/d2/98/d191289a877a34c68aa67e05179521e060f96394a3e9336be6
Successfully built sentence-transformers
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


In [11]:
from google.colab import auth
import json
import nltk
import numpy as np
from oauth2client.client import GoogleCredentials
import os
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import PyPDF2
import requests
from sentence_transformers import SentenceTransformer

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Earnings Call Data Ingestion

In [ ]:
symbols = ['AAPL', 'MSFT', 'GOOGL', 'XOM', 'CVX']

In [ ]:
def create_earnings_calls_list(symbols):
  earnings_calls = []

  for symbol in symbols:

    #api to get all transcripts per the symbol
    symbol_api_req = requests.get('https://finnhub.io/api/v1/stock/transcripts?symbol='+symbol+'&token=c2uvbbqad3i9mrpuqq00')
    
    for t in symbol_api_req.json().get("transcripts"):
      #api to get speech text, speaker, and other details per transcript
      transcript_api_req = requests.get('https://finnhub.io/api/v1/stock/transcripts?id='+t.get("id")+'&token=c2uvbbqad3i9mrpuqq00')
      resulting_json = transcript_api_req.json()
      
      #build name-designation loopup for each transcript
      name_designation_lookup = {}
      if("participant" in resulting_json.keys()):
        for p in resulting_json.get("participant"):
          name_designation_lookup[p.get("name")] = p.get("description") 
      #build dictionary to store speech and speaker details
      if("transcript" in resulting_json.keys()):
        for transcript in resulting_json.get("transcript"):
          if (transcript.get("name")!="Operator"):
            #speeches = []
            #speeches.append({})
          #save every transcript and speeches per transcript with symbol key in earnings_calls_dictionary
            earnings_calls.append({"symbol": symbol,
                                  "quarter": resulting_json.get("quarter"),
                                  "year": resulting_json.get("year"),
                                  "speaker_desig":name_designation_lookup.get(transcript.get("name"),"unknown"), 
                                  "speaker_name":transcript.get("name"),
                                  "speech":transcript.get("speech")})
  return earnings_calls

In [ ]:
earnings_call_df = pd.DataFrame(create_earnings_calls_list(symbols))

In [ ]:
earnings_call_df.head(20)

,symbol,quarter,year,speaker_desig,speaker_name,speech
0,AAPL,2,2021,Director of Investor Relations and Corporate F...,Tejas Gala,[Thank you. Good afternoon and thank you for j...
1,AAPL,2,2021,Chief Executive Officer,Tim Cook,"[Thanks, Tejas. Good afternoon, everyone, and ..."
2,AAPL,2,2021,Chief Financial Officer,Luca Maestri,"[Thank you, Tim. Good afternoon, everyone. We ..."
3,AAPL,2,2021,Director of Investor Relations and Corporate F...,Tejas Gala,"[Thank you, Luca. [Operator Instructions] Oper..."
4,AAPL,2,2021,Cross Research,Shannon Cross,"[Tim, I had sort of a big picture question on ..."
5,AAPL,2,2021,Chief Executive Officer,Tim Cook,"[Sure, Shannon. We saw double-digit increases ..."
6,AAPL,2,2021,Cross Research,Shannon Cross,"[Okay. And then, Luca, can you talk about gros..."
7,AAPL,2,2021,Chief Financial Officer,Luca Maestri,"[Yes, Shannon. Yes, we did 42.5% during March,..."
8,AAPL,2,2021,Evercore,Amit Daryanani,"[I have two as well. First one, just on servic..."
9,AAPL,2,2021,Chief Financial Officer,Luca Maestri,"[So Amit, the -- our Services business did bet..."


Join the field speech, which is a list that I believe is always of lenth 1, and make it a string

In [ ]:
earnings_call_df['speech'] = earnings_call_df.speech.str.join('')

Not used but code here works on one ticker, AAPL, just to document the process

In [ ]:
print("length of first speech:", len(temp_df['speech2'][0]))
print("length of entire dataframe:", len(earnings_call_df))
print("Unique symbols:", earnings_call_df.symbol.unique())


1123


22315

In [ ]:
earnings_call_df.symbol.unique()

array(['AAPL', 'MSFT', 'XOM', 'CVX'], dtype=object)

In [ ]:
aapl_one_df = earnings_call_df[(earnings_call_df.symbol=='AAPL') & (earnings_call_df.year==2014) & (earnings_call_df.quarter==2)]

In [ ]:
len(aapl_one_df)

42

In [ ]:
len(aapl_one_df.iloc[0,5])

1089

In [ ]:
temp_count = 0
for row in range(len(aapl_one_df)):
    temp_count += len(aapl_one_df.iloc[row,5])
print(temp_count)

47014


In [ ]:
max(earnings_call_df.speech.apply(len))

64462

In [ ]:
aapl_one_merged_df = aapl_one_df[['symbol','quarter', 'year', 'speech']].copy()
aapl_one_merged_df['speech'] = aapl_one_merged_df.groupby(['symbol','quarter', 'year'])['speech'].transform(lambda x: ' '.join(x))
aapl_one_merged_df = aapl_one_merged_df[['symbol','quarter', 'year', 'speech']].drop_duplicates()

In [ ]:
len(aapl_one_merged_df.speech.iloc[0])
#aapl_one_merged_df.head()

47055

# Sentence Relevance

In [ ]:
compare_sentences = ["In order to achieve these stated goals, , with the need to transition to a net zero carbon economy to stabilize global temperatures.",
                     "Broader societal role and considering value creation and erosion across multiple capitals: natural, social, human, financial, intellectual and manufactured capital, hunger and access to healthcare and education, equitable world",
                     "I look forward to working with peers from organizations that span multiple sectors and geographies. Working together at A4S, I believe we can work together, share best practices and come up with practical solutions that allow us to move our organizations – and encourage others – toward more sustainable outcomes."
                     ]

In [ ]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
def sort_sents_by_desc_order_relevance(earnings_call_df, compare_sentences):
  sorted_sents_for_speech = []
  sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
  for speech_index, speech in earnings_call_df.speech.items():
    sentences = nltk.sent_tokenize(speech[0])
    
    tuples_of_sents_sim = [(sent, 
                        sum([cosine(sbert_model.encode([compare_sent])[0], 
                                                sbert_model.encode([sent])[0])for compare_sent in compare_sentences])/len(compare_sentences))
                                                for sent in sentences]
    tuples_of_sents_sim.sort(key=lambda x:x[1], reverse=True)
    sorted_sents_for_speech.append(" ".join([x[0] for x in tuples_of_sents_sim]))
  return sorted_sents_for_speech

In [ ]:
limited_earnings_call_df = earnings_call_df[:10]

In [ ]:
sort_sents_by_desc_order_relevance(limited_earnings_call_df, compare_sentences)

["Please note that some of the information you'll hear during our discussion today will consist of forward-looking statements, including, without limitation, those regarding revenue, gross margin, operating expenses, other income and expense, taxes, capital allocation and future business outlook, including the potential impact of COVID-19 on the Company's business results of operation. These statements involve risks and uncertainties that may cause actual results or trends to differ materially from our forecast. After that, we'll open the call to questions from analysts. I'd now like to turn over the call to Tim for introductory remarks. For more information, please refer to the risk factors discussed in Apple's most recently filed annual report on Form 10-K and the Form 8-K filed with the SEC today, along with the associated press release. Speaking first today is Apple's CEO, Tim Cook, and he'll be followed by CFO, Luca Maestri. Good afternoon and thank you for joining us. Thank you. 

In [ ]:
limited_earnings_call_df["relevanceOrderedSents"] = sort_sents_by_desc_order_relevance(limited_earnings_call_df, compare_sentences)

NameError: ignored

In [ ]:
limited_earnings_call_df.head(20)

NameError: ignored

# Sustainaibility Reports

In [12]:
#ADF Reports
adfreports = []

In [13]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [14]:
# local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data/a4s')
try:
  os.makedirs(local_download_path)
except: pass

In [15]:
# 2. Auto-iterate using the query syntax
file_list = drive.ListFile(
    {'q': "'1wjLj01vGtA6zDxulEU9pxH9_dZcGkTl5' in parents"}).GetList()

In [16]:
print(file_list[0])

GoogleDriveFile({'kind': 'drive#file', 'id': '1XD3afMWpx_HWqATn6mJuPOUl2wdV65Lc', 'etag': '"MTYyNDkyNjYxMzAwMA"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1XD3afMWpx_HWqATn6mJuPOUl2wdV65Lc', 'webContentLink': 'https://drive.google.com/uc?id=1XD3afMWpx_HWqATn6mJuPOUl2wdV65Lc&export=download', 'alternateLink': 'https://drive.google.com/file/d/1XD3afMWpx_HWqATn6mJuPOUl2wdV65Lc/view?usp=drivesdk', 'embedLink': 'https://drive.google.com/file/d/1XD3afMWpx_HWqATn6mJuPOUl2wdV65Lc/preview?usp=drivesdk', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/application/pdf', 'thumbnailLink': 'https://lh3.googleusercontent.com/CDJd_feJeCUMyMrGvT7ECGazrQ4TDeqAP5bAMxr0Vehqe4aYSyQl1HisAYrYQ9ZA1wK_5Io0FNzR6Zk=s220', 'title': '12567_FFTF Rabobank case study v1.pdf.downloadasset.pdf', 'mimeType': 'application/pdf', 'labels': {'starred': False, 'hidden': False, 'trashed': False, 'restricted': False, 'viewed': True}, 'copyRequiresWriterPermission': False, 'createdDate': '2021-

In [17]:
for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)
  
  # creating a pdf file object
  pdfFileObj = open(fname, 'rb')

  # creating a pdf reader object
  pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
  
  # printing number of pages in pdf file
  print(pdfReader.numPages)
  
  # creating a page object
  pageObj = pdfReader.getPage(0)
  
  # extracting text from page
  text = [pdfReader.getPage(i).extractText() for i in range(0, pdfReader.getNumPages())]
  adfreports.append({"resource_name":f['title'], "text":"\n".join(text)})
  
  # closing the pdf file object
  pdfFileObj.close()

  #print('downloading to {}'.format(fname))
  #f_ = drive.CreateFile({'id': f['id']})
  #f_.GetContentFile(fname)

title: 12567_FFTF Rabobank case study v1.pdf.downloadasset.pdf, id: 1XD3afMWpx_HWqATn6mJuPOUl2wdV65Lc
downloading to /root/data/a4s/12567_FFTF Rabobank case study v1.pdf.downloadasset.pdf


4
title: A4S Essential Guide to Valuations and Climate Change.pdf.downloadasset.pdf, id: 1d68YLejIXpt5MdbdyQLyz8r4_HEeS6l_
downloading to /root/data/a4s/A4S Essential Guide to Valuations and Climate Change.pdf.downloadasset.pdf
104


In [18]:
adfreportsdf = pd.DataFrame(adfreports)

In [19]:
adfreportsdf

,resource_name,text
0,12567_FFTF Rabobank case study v1.pdf.download...,www.˜nanceforthefuture.org\nIn partnership wit...
1,A4S Essential Guide to Valuations and Climate ...,ESSENTIAL GUIDE TO \nVALUATIONS AND CLIMATE \...


In [20]:
adfreportsdf.text[1]

'ESSENTIAL GUIDE TO  \nVALUATIONS AND CLIMATE \n\nCHANGE\nA framework to assess the impact of climate change \non business valuations\nEssential Guide to \nVALUATIONS AND \n\nCLIMATE CHANGE\nIntroduction\nAppendices 1-4\nAppendix 5 Œ Case studies\nAuthority and scope of \n the framework\n\nClimate change valuation \nframework\n\nA4S Essential Guide to Valuations and Climate Change \n         \n \n \n \n \n    2Introduction\nAutorité et champ \nd™application du cadre\nCadre d™évaluation axé \nsur les changements \n\nclimatiques\nAnnexes 1-4\n\nAnnexe 5 Œ Études de cas\nGUIDE ESSENTIEL SUR \nLES ÉVALUATIONS ET LES \n\nCHANGEMENTS CLIMATIQUES\nRÉSEAU\n DE LEADERSHIP \nDES CHEFS DES \nFINANCES DE \nL™ADC\nIntroduction\nAuthority and scope of \n \nthe framework\nClimate change valuation \nframework\nAppendices 1-4\n\nAppendix 5 Œ Case studies\nEssential Guide to \nVALUATIONS AND \n\nCLIMATE CHANGE\nNAVIGATING THIS GUIDE\nWHAT IS INCLUDED\n\n\nŁForeword from David McGraw,  \n\nCFO, OTPP\nŁEx

In [21]:
esgRAWtextfile = open(r"esgRAWtext.txt","w")

esgRAWtextfile.writelines(adfreportsdf.text.to_numpy())

# ESG BERT Transformer

In [4]:
!pip install sentencepiece
!git clone https://github.com/google-research/bert

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 328.28 KiB | 4.00 MiB/s, done.
Resolving deltas: 100% (182/182), done.


In [5]:
!pip install tensorflow-gpu==1.15.0
!pip install tensorflow==1.15.0

     |████████████████████████████████| 411.5MB 28kB/s 
     |████████████████████████████████| 3.8MB 34.6MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 512kB 35.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=a709811d9bb7d96d690c449a058988a80f5a03a2dd13787c96e0fc8ea45b73ad
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.5.0 has requirement gast==0.4.0, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement tensorflow-estimator<2.6.0,>=2.5.0rc0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing ins

In [ ]:
%tensorflow_version 1.x

In [7]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
import sentencepiece as spm

from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("bert")

from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder

auth.authenticate_user()
  
# configure logging
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s :  %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]

if 'COLAB_TPU_ADDR' in os.environ:
  log.info("Using TPU runtime")
  USE_TPU = True
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']

  with tf.Session(TPU_ADDRESS) as session:
    log.info('TPU address is ' + TPU_ADDRESS)
    # Upload credentials to TPU.
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    
else:
  log.warning('Not connected to TPU runtime')
  USE_TPU = False


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



2021-07-06 13:02:09,539 :  Using TPU runtime
2021-07-06 13:02:09,546 :  TPU address is grpc://10.18.117.138:8470


In [8]:
def normalize_text(text):
  # remove non-UTF
  text = text.encode("utf-8", "ignore").decode()
  # remove \n
  text = text.replace("\n"," ")
  # remove extra spaces
  text = ' '.join(text.split())
  return text

def count_lines(filename):
  count = 0
  with open(filename) as fi:
    for line in fi:
      count += 1
  return count

In [22]:
RAW_DATA_FPATH = "esgRAWtext.txt" #@param {type: "string"}
PRC_DATA_FPATH = "esgPROCESSEDtext.txt" #@param {type: "string"}

# apply normalization to the dataset
# this will take a minute or two

total_lines = count_lines(RAW_DATA_FPATH)
bar = Progbar(total_lines)

with open(RAW_DATA_FPATH,encoding="utf-8") as fi:
  with open(PRC_DATA_FPATH, "w",encoding="utf-8") as fo:
    for l in fi:
      fo.write(normalize_text(l)+"\n")
      bar.add(1)

6686/6686 [==============================] - 0s 4us/step


In [23]:
MODEL_PREFIX = "tokenizer" #@param {type: "string"}
VOC_SIZE = 2000 #@param {type:"integer"}
SUBSAMPLE_SIZE = 12800000 #@param {type:"integer"}
NUM_PLACEHOLDERS = 256 #@param {type:"integer"}

SPM_COMMAND = ('--input={} --model_prefix={} '
               '--vocab_size={} --input_sentence_size={} '
               '--shuffle_input_sentence=true ' 
               '--bos_id=-1 --eos_id=-1').format(
               PRC_DATA_FPATH, MODEL_PREFIX, 
               VOC_SIZE - NUM_PLACEHOLDERS, SUBSAMPLE_SIZE)

spm.SentencePieceTrainer.Train(SPM_COMMAND)

In [24]:
def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format(MODEL_PREFIX))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

Learnt vocab size: 1743
Sample tokens: ['▁Company', 'just', 'tud', '▁Revenue', 'ARE', '▁private', 'based', '▁strong', '▁Water', 'Institution']


In [25]:
def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token
        
bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
bert_vocab = ctrl_symbols + bert_vocab

bert_vocab += ["[UNUSED_{}]".format(i) for i in range(VOC_SIZE - len(bert_vocab))]
print(len(bert_vocab))

2000


In [26]:
VOC_FNAME = "esgvocab.txt" #@param {type:"string"}

with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [27]:
testcase = "Certain sectors are particularly vulnerable to climate risk.  For example, the TCFD recommendations state that companies engaged in fossil-fuel based industries, energy-intensive manufacturing, and transportation activities are most exposed to transition risks, while those engaged in agriculture, transportation and building infrastructure, insurance and tourism are more exposed to physical risks."
bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)
bert_tokenizer.tokenize(testcase)

2021-07-06 13:04:46,621 :  From /content/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



['c',
 '##er',
 '##t',
 '##ain',
 'sectors',
 'are',
 'particular',
 '##ly',
 'v',
 '##u',
 '##l',
 '##ne',
 '##ra',
 '##ble',
 'to',
 'climate',
 'risk',
 '[UNK]',
 'for',
 'example',
 '[UNK]',
 'the',
 't',
 '##c',
 '##f',
 '##d',
 'reco',
 '##m',
 '##me',
 '##nd',
 '##ations',
 'state',
 'that',
 'companies',
 'en',
 '##ga',
 '##ge',
 '##d',
 'in',
 'f',
 '##ossil',
 '[UNK]',
 'f',
 '##u',
 '##e',
 '##l',
 'based',
 'industri',
 '##es',
 '[UNK]',
 'energy',
 '[UNK]',
 'intensi',
 '##ve',
 'ma',
 '##n',
 '##u',
 '##f',
 '##act',
 '##ur',
 '##ing',
 '[UNK]',
 'and',
 'tra',
 '##n',
 '##s',
 '##p',
 '##ort',
 '##ation',
 'activities',
 'are',
 'mo',
 '##st',
 'exp',
 '##o',
 '##se',
 '##d',
 'to',
 'transition',
 'risks',
 '[UNK]',
 'wh',
 '##il',
 '##e',
 'th',
 '##o',
 '##se',
 'en',
 '##ga',
 '##ge',
 '##d',
 'in',
 'a',
 '##gricultur',
 '##e',
 '[UNK]',
 'tra',
 '##n',
 '##s',
 '##p',
 '##ort',
 '##ation',
 'and',
 'b',
 '##uild',
 '##ing',
 'infrastructure',
 '[UNK]',
 'insurance'

In [28]:
MAX_SEQ_LENGTH = 512 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param
MAX_PREDICTIONS = 75 #@param {type:"integer"}
DO_LOWER_CASE = False #@param {type:"boolean"}

PRETRAINING_DIR = "pretraining_data" #@param {type:"string"}
# controls how many parallel processes xargs can create
PROCESSES = 2 #@param {type:"integer"}

In [29]:
!mkdir ./shards
!split -a 4 -l 256000 -d $PRC_DATA_FPATH ./shards/shard_

In [30]:
XARGS_CMD = ("ls ./shards/ |"
             "xargs -n 1 -P {} -I{} "
             "python3 bert/create_pretraining_data.py "
             "--input_file=./shards/{} "
             "--output_file={}/{}.tfrecord "
             "--vocab_file={} "
             "--do_lower_case={} "
             "--max_predictions_per_seq={} "
             "--max_seq_length={} "
             "--masked_lm_prob={} "
             "--random_seed=34 "
             "--dupe_factor=5")

XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}', PRETRAINING_DIR, '{}', 
                             VOC_FNAME, DO_LOWER_CASE, 
                             MAX_PREDICTIONS, MAX_SEQ_LENGTH, MASKED_LM_PROB)
                             
tf.gfile.MkDir(PRETRAINING_DIR)
!$XARGS_CMD



W0706 13:05:00.541477 140220668987264 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0706 13:05:00.541745 140220668987264 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0706 13:05:00.542016 140220668987264 module_wrapper.py:139] From /content/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0706 13:05:00.549776 140220668987264 module_wrapper.py:139] From bert/create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W0706 13:05:00.550603 140220668987264 module_wrapper.py:139] From bert/create_pretraining_data.py:446: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:*** Reading from input files ***

In [31]:
BUCKET_NAME = "esgbert_resourses" #@param {type:"string"}
MODEL_DIR = "esgbert_model" #@param {type:"string"}
tf.gfile.MkDir(MODEL_DIR)
!gsutil -m cp -r $MODEL_DIR $PRETRAINING_DIR gs://$BUCKET_NAME
if not BUCKET_NAME:
  log.warning("WARNING: BUCKET_NAME is not set. "
              "You will not be able to train the model.")

Copying file://pretraining_data/shard_0000.tfrecord [Content-Type=application/octet-stream]...
/ [1/1 files][ 14.5 MiB/ 14.5 MiB] 100% Done                                    
Operation completed over 1 objects/14.5 MiB.                                     


In [32]:
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  account: srishtimehra@berkeley.edu

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you would like to use to perform operations for 
this configuration:
 [1] srishtimehra@berkeley.edu
 [2] Log in with a new account
Please enter your numeric choice:  1

You are logged in as: [srishtimehra@berkeley.edu].

Pick cloud project to use: 


In [32]:
bert_base_config = {
  "attention_probs_dropout_prob": 0.1, 
  "directionality": "bidi", 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 768, 
  "initializer_range": 0.02, 
  "intermediate_size": 3072, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 12, 
  "num_hidden_layers": 12, 
  "pooler_fc_size": 768, 
  "pooler_num_attention_heads": 12, 
  "pooler_num_fc_layers": 3, 
  "pooler_size_per_head": 128, 
  "pooler_type": "first_token_transform", 
  "type_vocab_size": 2, 
  "vocab_size": VOC_SIZE
}

with open("{}/bert_config.json".format(MODEL_DIR), "w") as fo:
  json.dump(bert_base_config, fo, indent=2)
  
with open("{}/{}".format(MODEL_DIR, VOC_FNAME), "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [33]:
if BUCKET_NAME:
  !gsutil -m cp -r $MODEL_DIR $PRETRAINING_DIR gs://$BUCKET_NAME

Copying file://esgbert_model/bert_config.json [Content-Type=application/json]...
Copying file://esgbert_model/esgvocab.txt [Content-Type=text/plain]...
Copying file://pretraining_data/shard_0000.tfrecord [Content-Type=application/octet-stream]...
/ [3/3 files][ 14.5 MiB/ 14.5 MiB] 100% Done                                    
Operation completed over 3 objects/14.5 MiB.                                     


In [39]:
BUCKET_NAME = "esgbert_resourses" #@param {type:"string"}
MODEL_DIR = "esgbert_model" #@param {type:"string"}
PRETRAINING_DIR = "pretraining_data" #@param {type:"string"}
VOC_FNAME = "esgvocab.txt" #@param {type:"string"}

# Input data pipeline config
TRAIN_BATCH_SIZE = 128 #@param {type:"integer"}
MAX_PREDICTIONS = 75 #@param {type:"integer"}
MAX_SEQ_LENGTH = 512 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param

# Training procedure config
EVAL_BATCH_SIZE = 64
LEARNING_RATE = 2e-5
TRAIN_STEPS = 100 #@param {type:"integer"}
SAVE_CHECKPOINTS_STEPS = 25 #@param {type:"integer"}
NUM_TPU_CORES = 8

if BUCKET_NAME:
  BUCKET_PATH = "gs://{}".format(BUCKET_NAME)
else:
  BUCKET_PATH = "."

BERT_GCS_DIR = "{}/{}".format(BUCKET_PATH, MODEL_DIR)
DATA_GCS_DIR = "{}/{}".format(BUCKET_PATH, PRETRAINING_DIR)

VOCAB_FILE = os.path.join(BERT_GCS_DIR, VOC_FNAME)
CONFIG_FILE = os.path.join(BERT_GCS_DIR, "bert_config.json")

INIT_CHECKPOINT = tf.train.latest_checkpoint(BERT_GCS_DIR)

bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)
input_files = tf.gfile.Glob(os.path.join(DATA_GCS_DIR,'*tfrecord'))

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))
log.info("Using {} data shards".format(len(input_files)))

2021-07-06 13:06:38,646 :  Using checkpoint: None
2021-07-06 13:06:38,649 :  Using 1 data shards


In [41]:
model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=LEARNING_RATE,
      num_train_steps=TRAIN_STEPS,
      num_warmup_steps=10,
      use_tpu=USE_TPU,
      use_one_hot_embeddings=True)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=BERT_GCS_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

#run_config = tf.estimator.RunConfig(
 #   model_dir=BERT_GCS_DIR,
  #  save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

estimator = tf.contrib.tpu.TPUEstimator(
#estimator = tf.estimator.Estimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)
  
train_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS,
        is_training=True)

2021-07-06 13:06:49,158 :  Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f0e85831050>) includes params argument, but params are not passed to Estimator.
2021-07-06 13:06:49,164 :  Using config: {'_model_dir': 'gs://esgbert_resourses/esgbert_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 25, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.18.117.138:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0e85647e90

In [42]:
estimator.train(input_fn=train_input_fn, max_steps=TRAIN_STEPS)

2021-07-06 13:06:53,216 :  Querying Tensorflow master (grpc://10.18.117.138:8470) for TPU system metadata.
2021-07-06 13:06:53,227 :  Found TPU system:
2021-07-06 13:06:53,228 :  *** Num TPU Cores: 8
2021-07-06 13:06:53,235 :  *** Num TPU Workers: 1
2021-07-06 13:06:53,236 :  *** Num TPU Cores Per Worker: 8
2021-07-06 13:06:53,247 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 15916867275321068859)
2021-07-06 13:06:53,250 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 18062280619181287568)
2021-07-06 13:06:53,251 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4625725992863806867)
2021-07-06 13:06:53,253 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 9077909587501769289)
2021-07-06 13:06:53,254 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device

In [43]:
estimator.export_saved_model(MODEL_DIR)

TypeError: ignored